In [111]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import scipy as scp
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import sklearn
from sklearn.linear_model import LinearRegression
import os
import Levenshtein

In [112]:
pd.options.display.max_rows = 600
pd.options.display.max_columns = 85

#### read in geocoded

In [113]:
df = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/new_totalprojects.csv')
df.head()

,ObjectID,Loc_name,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,ARC_Address,ARC_Address2,ARC_Address3,ARC_Neighborhood,ARC_City,ARC_Subregion,ARC_Region,ARC_Postal,ARC_PostalExt,ARC_CountryCode,Parcel_ID,Address,City_1,State,XY_Coordinates,XY_Coordinates_X,XY_Coordinates_Y,Plan_Year,IDIS_Project,Project,IDIS_Activity_ID,Activity_Status,Program,Funded_Amount,Draw_Amount,Balance
0,1,World,M,100.0,A,"1707 Edgerly Ave, Albany, Georgia, 31707","1707 Edgerly Ave, Albany, GA, 31707, USA",1707 Edgerly Ave,PointAddress,NaN,NaN,"1707 Edgerly Ave, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1707.0,NaN,NaN,NaN,L,NaN,NaN,Edgerly,Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1707 Edgerly Ave,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,4208.0,USA,ENG,0.0,-84.191282,31.573277,-84.191292,31.573530,-84.192292,-84.190292,31.572530,31.574530,NaN,1707 Edgerly Avenue,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000Q/00025/009,1707 Edgerly Avenue,Albany,GA,"31.5735022, -84.191293",-84.191293,31.573502,2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,2,World,M,98.7,A,"703 Dartmouth Ln, Albany, Georgia, 31705","703 Dartmouth Ln, Albany, GA, 31705, USA",703 Dartmouth Ln,PointAddress,NaN,NaN,"703 Dartmouth Ln, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,703.0,NaN,NaN,NaN,R,NaN,NaN,Dartmouth,Ln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,703 Dartmouth Ln,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,1211.0,USA,ENG,0.0,-84.136801,31.588103,-84.136802,31.588326,-84.137802,-84.135801,31.587326,31.589326,NaN,703 Dartmouth,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000U/00006/002,703 Dartmouth,Albany,GA,"31.5882777, -84.1367938",-84.136794,31.588278,2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,3,World,M,100.0,A,"1115 W Waddell Ave, Albany, Georgia, 31707","1115 W Waddell Ave, Albany, GA, 31707, USA",1115 W Waddell Ave,PointAddress,NaN,NaN,"1115 W Waddell Ave, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1115.0,NaN,NaN,NaN,L,W,NaN,Waddell,Ave,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1115 W Waddell Ave,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,5115.0,USA,ENG,0.0,-84.178174,31.566717,-84.178184,31.567059,-84.179183,-84.177183,31.566059,31.568059,NaN,1115 W. Waddell Avenue,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000H/00047/005,1115 W. Waddell Avenue,Albany,GA,"31.5669637, -84.1782082",-84.178208,31.566964,2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,4,World,M,98.7,A,"313 Forrestal Ln, Albany, Georgia, 31705","313 Forrestal Ln, Albany, GA, 31705, USA",313 Forrestal Ln,PointAddress,NaN,NaN,"313 Forrestal Ln, Albany, Georgia, 31705",NaN,NaN,20.0,NaN,313.0,NaN,NaN,NaN,L,NaN,NaN,Forrestal,Ln,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,313 Forrestal Ln,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31705.0,2508.0,USA,ENG,0.0,-84.082134,31.578470,-84.082442,31.578579,-84.083442,-84.081441,31.577579,31.579579,NaN,313 Forrestal,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,0000X/00003/034,313 Forrestal,Albany,GA,"31.5785109, -84.0824103",-84.082410,31.578511,2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,5,World,M,100.0,A,"1125 Joel Dr, Albany, Georgia, 31707","1125 Joel Dr, Albany, GA, 31707, USA",1125 Joel Dr,PointAddress,NaN,NaN,"1125 Joel Dr, Albany, Georgia, 31707",NaN,NaN,20.0,NaN,1125.0,NaN,NaN,NaN,L,NaN,NaN,Joel,Dr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1125 Joel Dr,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31707.0,5021.0,USA,ENG,0.0,-84.207624,31.561222,-84.207901,31.561119,-84.208901,-84.206901,31.560119,31.562119,NaN,11

In [114]:
df.tail(10)

,ObjectID,Loc_name,Status,Score,Match_type,Match_addr,LongLabel,ShortLabel,Addr_type,Type,PlaceName,Place_addr,Phone,URL,Rank,AddBldg,AddNum,AddNumFrom,AddNumTo,AddRange,Side,StPreDir,StPreType,StName,StType,StDir,BldgType,BldgName,LevelType,LevelName,UnitType,UnitName,SubAddr,StAddr,Block,Sector,Nbrhd,District,City,MetroArea,Subregion,Region,RegionAbbr,Territory,Zone,Postal,PostalExt,Country,LangCode,Distance,X,Y,DisplayX,DisplayY,Xmin,Xmax,Ymin,Ymax,ExInfo,ARC_Address,ARC_Address2,ARC_Address3,ARC_Neighborhood,ARC_City,ARC_Subregion,ARC_Region,ARC_Postal,ARC_PostalExt,ARC_CountryCode,Parcel_ID,Address,City_1,State,XY_Coordinates,XY_Coordinates_X,XY_Coordinates_Y,Plan_Year,IDIS_Project,Project,IDIS_Activity_ID,Activity_Status,Program,Funded_Amount,Draw_Amount,Balance
840,841,World,M,100.00,A,"1704 S Slappey Blvd, Albany, Georgia, 31701","1704 S Slappey Blvd, Albany, GA, 31701, USA",1704 S Slappey Blvd,PointAddress,NaN,NaN,"1704 S Slappey Blvd, Albany, Georgia, 31701",NaN,NaN,20.0,NaN,1704.0,NaN,NaN,NaN,R,S,NaN,Slappey,Blvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1704 S Slappey Blvd,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31701.0,2648.0,USA,ENG,0.0,-84.176582,31.553397,-84.175974,31.553298,-84.177974,-84.173974,31.551298,31.555298,NaN,1704 S Slappey Blvd,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,NaN,1704 S Slappey Blvd,Albany,GA,NaN,NaN,NaN,2017,7,Public Service,1481,Open,CDBG,10000.00,10000.00,0.0
841,842,World,M,100.00,A,"314 N Jefferson St, Albany, Georgia, 31701","314 N Jefferson St, Albany, GA, 31701, USA",314 N Jefferson St,StreetAddress,NaN,NaN,"314 N Jefferson St, Albany, Georgia, 31701",NaN,NaN,20.0,NaN,314.0,312.0,326.0,312-326,R,N,NaN,Jefferson,St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,314 N Jefferson St,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31701.0,2556.0,USA,ENG,0.0,-84.155958,31.581165,-84.155958,31.581165,-84.156958,-84.154958,31.580165,31.582165,NaN,314 N Jefferson St,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,NaN,314 N Jefferson St,Albany,GA,NaN,NaN,NaN,2017,7,Public Service,1478,Open,CDBG,15000.00,15000.00,0.0
842,843,World,M,100.00,A,"900 Lippitt Dr, Albany, Georgia, 31701","900 Lippitt Dr, Albany, GA, 31701, USA",900 Lippitt Dr,PointAddress,NaN,NaN,"900 Lippitt Dr, Albany, Georgia, 31701",NaN,NaN,20.0,NaN,900.0,NaN,NaN,NaN,R,NaN,NaN,Lippitt,Dr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,900 Lippitt Dr,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31701.0,3511.0,USA,ENG,0.0,-84.171200,31.559000,-84.170110,31.557267,-84.172111,-84.168110,31.555267,31.559267,NaN,900 Lippitt Dr,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,NaN,900 Lippitt Dr,Albany,GA,NaN,NaN,NaN,2017,7,Public Service,1480,Open,CDBG,25000.00,25000.00,0.0
843,844,World,M,98.86,A,"405 Johnny W Williams Rd, Albany, Georgia, 31701","405 Johnny W Williams Rd, Albany, GA, 31701, USA",405 Johnny W Williams Rd,PointAddress,NaN,NaN,"405 Johnny W Williams Rd, Albany, Georgia, 31701",NaN,NaN,20.0,NaN,405.0,NaN,NaN,NaN,L,NaN,NaN,Johnny W Williams,Rd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,405 Johnny W Williams Rd,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31701.0,4204.0,USA,ENG,0.0,-84.155913,31.540045,-84.155913,31.540257,-84.156913,-84.154913,31.539257,31.541257,NaN,405 Johnny Williams Rd,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,NaN,405 Johnny Williams Rd,Albany,GA,NaN,NaN,NaN,2017,1,Disposition,1521,Open,CDBG,11259.61,6924.45,4335.0
844,845,World,M,100.00,A,"101 S Front St, Albany, Georgia, 31701","101 S Front St, Albany, GA, 31701, USA",101 S Front St,PointAddress,NaN,NaN,"101 S Front St, Albany, Georgia, 31701",NaN,NaN,20.0,NaN,101.0,NaN,NaN,NaN,L,S,NaN,Front,St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,101 S Front St,NaN,NaN,NaN,NaN,Albany,NaN,Dougherty County,Georgia,GA,NaN,NaN,31701.0,2801.0,USA,ENG,0.0,-84.149063,31.576212,-84.149721,31.576212,-84.150721,-84.148721,31.575212,31.577212,NaN,101 S Front St,NaN,NaN,NaN,Albany,NaN,GA,NaN,NaN,NaN,NaN,101 S Front St,Albany,GA,NaN,NaN,NaN,2017,5,Section 108 Loan Program,1464,Open,CDBG,

#### read in originals

In [115]:
xlsx = pd.ExcelFile('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/All CED Funded Projects 2007-2017.xlsx')
df1_original = pd.read_excel(xlsx, 'HOME Funded Projects')
df2_original = pd.read_excel(xlsx, 'CDBG Funded Projects')
df_original = pd.concat([df1_original, df2_original])
df_original.index = range(850)
df_original.head()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 Edgerly Avenue,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,0000U/00006/002,703 Dartmouth,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,0000H/00047/005,1115 W. Waddell Avenue,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,0000X/00003/034,313 Forrestal,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,00240/00003/024,1125 Joel Drive,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0


#### fix new

In [116]:
df1_fixed = pd.read_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/HOME_2007-2017_fixed_address.csv')
df2_fixed = pd.read_csv('/Users/william/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/CDBG_2007-2017_fixed_address.csv')

In [117]:
df1_fixed = df1_fixed.drop('Unnamed: 0', axis = 1)
df2_fixed = df2_fixed.drop('Unnamed: 0', axis = 1)

In [118]:
series_funded = df1_fixed['Funded Amount']
series_funded;

In [119]:
df1_fixed.head()

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 EDGERLY AVE,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,0000U/00006/002,703 DARTMOUTH LN,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0
2,0000H/00047/005,1115 W WADDELL AVE,Albany,GA,"31.5669637, -84.1782082",2007,6,Homeowner Rehabilitation,610,Completed,HOME,28678.19,28678.19,0.0
3,0000X/00003/034,313 FORRESTAL LN,Albany,GA,"31.5785109, -84.0824103",2007,6,Homeowner Rehabilitation,633,Completed,HOME,34694.47,34694.47,0.0
4,00240/00003/024,1125 JOEL DR,Albany,GA,"31.5611455, -84.2078905",2007,6,Homeowner Rehabilitation,600,Completed,HOME,30756.96,30756.96,0.0


In [120]:
df2_fixed.columns

Index(['Parcel ID', 'Address', 'City', 'State', 'XY Coordinates', 'Plan Year',
       'IDIS Project', 'Project', 'IDIS Activity ID', 'Activity Status',
       'Program', 'Funded Amount', 'Draw Amount', 'Balance'],
      dtype='object')

#### comparisons

In [121]:
df1 = df[df['Program'] == "HOME"]
df2 = df[df['Program'] == "CDBG"]

In [122]:
df_utilities2012 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2012.csv')
df_utilities2013 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2013.csv')
df_utilities2014 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2014.csv')
df_utilities2015 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2015.csv')
df_utilities2016 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2016.csv')
df_utilities2017 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2017.csv')
df_utilities2018 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2018.csv')
df_utilities2019 = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/Totals2019.csv')
df_utilities_list = [df_utilities2012,df_utilities2013,df_utilities2014,df_utilities2015,
                    df_utilities2016,df_utilities2017,df_utilities2018,df_utilities2019]

In [123]:
list1 = []
list2 = []
for df_u in df_utilities_list:
    list1.append(df_u['Premise Address'])
for series_u in list1:
    for row in series_u:
        list2.append(row)
series_utilities = pd.Series(list2)

In [15]:
#series_utilities = df_utilities2012['Premise Address']

In [124]:
series_housing_home = df2_fixed['Address']

In [125]:
addresses = []
for address in series_housing:
    address = address.strip()
    new_address = ""
    address_split = address.split(' ')
    length = len(address_split)
    for i in range(len(address_split)):
        if i == 1:
            if not (address_split[i] == 'N' or address_split[i] == 'S'
            or address_split[i] == 'E' or address_split[i] == 'W'):    
                new_address += ' '
        if i == length-1:
            new_address += ' '
        new_address += (address_split[i] + ' ')
    new_address = new_address.strip()
    addresses.append(new_address)
series_housing = pd.Series(addresses)

NameError: name 'series_housing' is not defined

In [18]:
series_unmatched = series_housing[~(series_housing.isin(series_utilities.str.upper()))]

NameError: name 'series_housing' is not defined

In [126]:
series_unmatched

NameError: name 'series_unmatched' is not defined

waddell should be w waddell <br>
e fourth should be E 4th <br>
highland should be w highland <br>
1706 s jefferson seems like it doesn't exist? <br>
highland again <br>
riverview should be s riverview <br>
2002 skylane might need to be 2002 n skylane? <br>
shadowlawn should be s shadowlawn <br>
629 harmon might need to be 620 <br>
823 shadowlawn should be s shadowlawn dr <br>
Unknown address?? maybe drop the record--maybe useful to still have <br>
no 603 n jefferson st <br> 
no 400 ebony ln (there is a ebony ln proj with same id but not 400) <br>
no 638 w highland ave <br>
1400 whitney should be 1400 w whitney <br>
307  SHADOWLAWN  DR should be s shadowlawn <br>
1602  ESTELLE  AVE should be st <br>
1918  HIGHLAND  AVE should be w highland <br>
2207  OXFORD  DR should be oxford rd <br>
700  COLQUITT  CR should be cir <br>
701  WILLIARD  AVE should be willard <br>
425 W SOCIETY  AVE might need to be 426 (change to 106 N Monroe ST) <br>
302 S CASON  AVE should be cason st <br>
SUPPRESSED   ADDRESS ?? <br>
225 S JACKSON  ST might be 2205? DNE IRL ANYMORE<br>
1010 W LINCOLN  AVE not sure what number it should be DNE IRL DEMOLITION PROJECT<br>
1105  JORDAN  AVE should be st <br>
1120  LINCOLN  AVE should be w lincoln <br>
estelle ave should be st <br>
1401 E CAMPBELL ST/108 CARROLL  ST????? <br>
1501  WADDELL  AVE should be w waddell <br>
1819  GREENVALE  AVE should be rd <br>
1907  GROVELAND  DR should be rd <br>
1914  HIGHLAND  AVE should be w highland <br>
2016  MLK JR  DR should be martin luther king jr dr i think <br>
219  TALAFAX  ST should be rd <br>
2572  CRESCENT  DR should be cresent dr <b> has to be changed in utilities </b> <br>
303  INDEPENDENCE  AVE should be dr <br>
424  HEARD  AVE might be 423? DNE IRL DEMOLITION PROJECT <br>
4835  IMPALA  LN not listed <br>
504  MAGNOLIA  ST should be s magnolia <br>
608  RESIDENCE  AVE should be W residence also maybe diff number DNE IRL DEMOLITION PROJECT <br>
614  ROADWAY  ST should be rd? SHOULD BE FIXED IN UTILITIES<br>
706 E TOWNE  RD should be easttown rd i think <br>
719  BARKLEY  AVE should be blvd <br>
804  HIGHLAND  AVE should be w highland <br>
919  LINCOLN  AVE should be w lincoln <br>
1210  TENTH  AVE should be 10th <br>
743 E TOWN  RD should be easttowne? <br>
1105  ST ANDREWS  DR should be saint andrews dr <br>
2004  WHITNEY  AVE should be 1004 maybe <br>
2011  COVES  CT should be cove ct <br>
732 W OGLETHORPE  BLVD might be 731 or 712 <br>
108  INGLESIDE  DR should be s ingleside dr <br>
2116  WADDELL  AVE should be w waddell <br>
1306  NEWTON  RD dne <br>
Oglethropes not sure why there are so many incorrect addresses on someone's end <br>
1302 E SECOND  AVE should be 2nd <br>
1817 W OGLETHORPE  BLVD dne <br>
1921  EDGERLY  DR should be ave <br>
202 N W  RD should be 202 N West  RD <br>
2020 N RIVERVIEW  CR should be dr <br>
214 N BROADAY  ST should be broadway <br>
2208  OXFORD  DR should be rd <br>
2536  CRESCENT  DR should be cresent <b> utilities change </b> <br>
502  LINCOLN  AVE not sure where it is <br>
516 W LINCOLN  AVE not sure <br>
607  COCHRAN  AVE might need to be 606 <br>
622  CEDAR  AVE dne <br>
703  GILBERT  LN dne <br>
422  LOIUS  AVE should be louis <br>
2301  GEORGE  ST nothing near this <br>
1012 W MERCER  AVE should be reg mercer <br>
1117  WADDELL  AVE should be w waddell <br>
1312  MOBILE  RD should be ave <br>
1934 S RIVERVIEW  CR should be cir <br>
2028 N RIVERVIEW  DR should be cir <br>
411  BONNYVIEW  AVE should be bonny view <br>
701  ANDOVER  LN dne <br>
3019  MARTIN LUTHER KING JR  DR dne <br>
NETHERLAND  LN something to do with block i think <br>
3201  WINTERWOOD  AVE dne <br>
706 W GORDON  AVE dne <br>
405  JOHNNY W WILLIAMS  RD dne <br>

In [20]:
temp_list = []
temp_list_u = []
address = "2016  MARTIN LUTHER KING JR  DR"
for u_address in series_utilities:
    if Levenshtein.distance(address, u_address.upper()) <= 2:
        temp_list.append(address)
        temp_list_u.append(u_address.upper())
temp_list_series = pd.Series(temp_list)
temp_list_u_series = pd.Series(temp_list_u)
temp_list_u_series

0        816  MARTIN LUTHER KING JR  DR
1        816  MARTIN LUTHER KING JR  DR
2        816  MARTIN LUTHER KING JR  DR
3        816  MARTIN LUTHER KING JR  DR
4        816  MARTIN LUTHER KING JR  DR
5        816  MARTIN LUTHER KING JR  DR
6       1916  MARTIN LUTHER KING JR  DR
7       1916  MARTIN LUTHER KING JR  DR
8       1916  MARTIN LUTHER KING JR  DR
9       1916  MARTIN LUTHER KING JR  DR
10      2003  MARTIN LUTHER KING JR  DR
11      2003  MARTIN LUTHER KING JR  DR
12      2003  MARTIN LUTHER KING JR  DR
13      2003  MARTIN LUTHER KING JR  DR
14      2005  MARTIN LUTHER KING JR  DR
15      2019  MARTIN LUTHER KING JR  DR
16      2019  MARTIN LUTHER KING JR  DR
17      2019  MARTIN LUTHER KING JR  DR
18      2023  MARTIN LUTHER KING JR  DR
19      2023  MARTIN LUTHER KING JR  DR
20      2023  MARTIN LUTHER KING JR  DR
21      2023  MARTIN LUTHER KING JR  DR
22      2023  MARTIN LUTHER KING JR  DR
23      2023  MARTIN LUTHER KING JR  DR
24      2023  MARTIN LUTHER KING JR  DR


In [21]:
temp_list_u_series;

#### issues with address so far

seems like when they add a cardinal direction, there is only one space between number, dir, and streetname, but still 2 between street and street type <br>
1118 whitney ave is not found in utilities -- fixed: should be w whitney<br>
greeenbrier ct in utilities not greenbriar <br>
estelle st in utilities not ave <br>
<b>2620 s madison is missing st in utilities data</b>
different s caroline ave's verified in pr05 not in utilities
1330 mobile road seems to be missing, there are mobile road addresses however --fixed should be avenue<br>
same with 504 magnolia st --fixed should be 504 s magnolia street <br>
1110 waddell should be west waddell <br>
520 cochran in utilities list but not 529 <br>
unclear about this but bonnyview is one word on google maps but two on the actual street sign. <br>
200s for Bonny view doesn't really seem to be in utilities data? <br>
apts are not listed in utilities data <br>
independence dr not ave in utilities data <br>
dawson rd r1 meaning? no r1 in utilities <br>
same with don cutler sr dr 90?. <br>
should be west highland ave <br>
typo in utilities should be 603 n haley st <br>
1112 peachtree ter not 1112a <br>
305 station crossing should be station crossing dr <br>
1902 south flintlock dr <br>
818 westview drive? does it exist? <br>
farrell ct not ferrell <br>
1003 holland dr is not existing yet? <br>
mclain not mcclain <br>
amsterdam ln not dr <br>
valencia st not dr <br>

seems like when they add a cardinal direction, there is only one space between number, dir, and streetname, but still 2 between street and street type <br>
1118 whitney ave is not found in utilities -- fixed: should be w whitney<br>
greeenbrier ct in utilities not greenbriar <br>
estelle st in utilities not ave <br>
<b>2620 s madison is missing st </b> in utilities data <br>
<b>different s caroline ave's verified in pr05 not in utilities </b>
1330 mobile road seems to be missing, there are mobile road addresses however --fixed should be avenue<br>
same with 504 magnolia st --fixed should be 504 s magnolia street <br>
1110 waddell should be west waddell <br>
520 cochran in utilities list but not 529 -- FIXED has to be fixed in utilities (listed as st) <br>
unclear about this but bonnyview is one word on google maps but two on the actual street sign. <br>
200s for Bonny view doesn't really seem to be in utilities data? ALSO no project listed? <br>
apts are not listed in utilities data <br>
independence dr not ave in utilities data <br>
dawson rd r1 meaning? no r1 in utilities <br>
same with don cutler sr dr 90?. <br>
should be west highland ave <br>
<b>typo in utilities should be 603 n haley st</b> <br>
1112 peachtree ter not 1112a <br>
305 station crossing should be station crossing dr <br>
1902 south flintlock dr <br>
818 westview drive? does it exist? <br>
farrell ct not ferrell <br>
1003 holland dr is not existing yet? <br>
mclain not mcclain <br>
amsterdam ln not dr <br>
valencia st not dr <br>

## editing apt and duplex

In [127]:
df_parsehome = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_home_v02.csv')
df_parsecdbg = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/fixed_cdbg.csv')

In [128]:
df_parsehome = df_parsehome.drop('Unnamed: 0', axis = 1)
df_parsecdbg = df_parsecdbg.drop('Unnamed: 0', axis = 1);

In [129]:
df_parsehome.head(2)

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,0000Q/00025/009,1707 EDGERLY AVE,Albany,GA,"31.5735022, -84.191293",2007,6,Homeowner Rehabilitation,618,Completed,HOME,27033.46,27033.46,0.0
1,0000U/00006/002,703 DARTMOUTH LN,Albany,GA,"31.5882777, -84.1367938",2007,6,Homeowner Rehabilitation,634,Completed,HOME,30185.60,30185.60,0.0


In [130]:
df_parsecdbg.head(2)

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
0,NaN,1008 E RESIDENCE AVE,Albany,GA,NaN,2007,7,Emergency Repairs,578,Completed,CDBG,5671.14,5671.14,0.0
1,NaN,1100 SWIFT ST,Albany,GA,NaN,2007,5,Energy Efficiency,619,Completed,CDBG,4580.04,4580.04,0.0


In [131]:
s_parsehome = df_parsehome['Address']
s_parsecdbg = df_parsecdbg['Address']

In [132]:
def minmaxseries(series):
    minNum = 10
    maxNum = -10
    minAdd = ""
    maxAdd = ""
    for address in series:
        full_address = address.split(' ')
        length = len(full_address)
        if length < minNum:
            minNum = length
            minAdd = address
        if length > maxNum:
            maxNum = length
            maxAdd = address
    print(minAdd)
    print(maxAdd)

In [133]:
minmaxseries(s_parsehome)
minmaxseries(s_parsecdbg)

1707 EDGERLY AVE
525 DON CUTLER SR DR APT 90
UNKNOWN ADDRESS
1926 MARTIN LUTHER KING JR DR


In [134]:
def containsAPT(series):
    apt_list = []
    for address in series:
        if "APT" in address:
            apt_list.append(address)
    return pd.Series(apt_list)

In [135]:
def containsDUP(series):
    dup_list = []
    for address in series:
        full_address = address.split(' ')
        length = len(full_address)
        for i in range(length - 1):
            word = full_address[i]
            if len(word) == 1:
                if not ((word == 'N') or (word == 'E')
                or (word == 'S') or (word == 'W')):
                    dup_list.append(address)
    return pd.Series(dup_list)

In [136]:
def clean(series):
    tmp_list = []
    for address in series:
        full_address = address.split(' ')
        length = len(full_address)
        if any(char.isdigit() for char in full_address[length-1]):
            tmp_list.append(address)
        else:
            tmp_list.append("---")
    return pd.Series(tmp_list)

In [137]:
clean(s_parsehome);

In [138]:
containsAPT(s_parsehome)
containsAPT(s_parsecdbg);

In [139]:
containsDUP(s_parsehome)
containsDUP(s_parsecdbg);

In [140]:
def splitter(series):
    apt_list = set(containsAPT(series))
    dup_list = set(containsDUP(series))
    sno_to_series = []
    snm_to_series = []
    stp_to_series = []
    apt_to_series = []
    dup_to_series = []

    for address in series:
        full_address = address.split()
        length = len(full_address)
        begin_index = 1
        end_index = length - 1
        #mutually exclusive apt and duplex
        if address == "UNKNOWN ADDRESS":
            sno_to_series.append("NA")
            snm_to_series.append("UNKNOWN ADDRESS")
            stp_to_series.append("NA")
            apt_to_series.append("NA")
            dup_to_series.append("NA")
            continue
        if address == "SUPPRESSED ADDRESS":
            sno_to_series.append("NA")
            snm_to_series.append("SUPPRESSED ADDRESS")
            stp_to_series.append("NA")
            apt_to_series.append("NA")
            dup_to_series.append("NA")
            continue
        if address in apt_list:
            apt_to_series.append(full_address[length-1])
            stp_to_series.append(full_address[length-3])
            end_index -= 2
        else:
            apt_to_series.append("NA")
            stp_to_series.append(full_address[length-1])
            
        if address in dup_list:
            dup_to_series.append(full_address[1])
            begin_index += 1
        else:
            dup_to_series.append("NA")
        sno_to_series.append(full_address[0])
        to_str = full_address[begin_index:end_index]
        word = ""
        for part in to_str:
            word += part + " "
        snm_to_series.append(word.strip())
    series_apt = pd.Series(apt_to_series)
    series_dup = pd.Series(dup_to_series)
    series_sno = pd.Series(sno_to_series)
    series_snm = pd.Series(snm_to_series)
    series_stp = pd.Series(stp_to_series)
    return (series_apt, series_dup, series_sno, series_snm, series_stp)

In [141]:
series_apt_home, series_dup_home, series_sno_home, series_snm_home, series_stp_home = splitter(s_parsehome)
series_apt_cdbg, series_dup_cdbg, series_sno_cdbg, series_snm_cdbg, series_stp_cdbg = splitter(s_parsecdbg)

In [142]:
series_stp_home[series_stp_home != "NA"];

In [143]:
series_snm_cdbg[series_snm_cdbg != "NA"];

In [144]:
home = [series_apt_home, series_dup_home, series_sno_home, series_snm_home, series_stp_home]
cdbg = [series_apt_cdbg, series_dup_cdbg, series_sno_cdbg, series_snm_cdbg, series_stp_cdbg]
for hom in home:
    print(hom.shape)
for cdb in cdbg:
    print(cdb.shape)

(286,)
(286,)
(286,)
(286,)
(286,)
(564,)
(564,)
(564,)
(564,)
(564,)


In [145]:
def checker(list_series):
    tmp_to_series = []
    for (apt, dup, sno, snm, stp) in zip(list_series[0],list_series[1],list_series[2],
                                         list_series[3],list_series[4]):
        word = ""
        if sno != "NA":
            word += sno + " "
        if dup != "NA":
            word += dup + " "
        word += snm + " "
        if stp != "NA":
            word += stp + " "
        if apt != "NA":
            word += "APT " + apt
        word = word.strip()
        tmp_to_series.append(word)
    return pd.Series(tmp_to_series)

In [146]:
s_parsehome[checker(home) != s_parsehome]

Series([], Name: Address, dtype: object)

In [147]:
s_parsecdbg[checker(cdbg) != s_parsecdbg]

Series([], Name: Address, dtype: object)

In [148]:
checker(cdbg)[checker(cdbg) != s_parsecdbg]

Series([], dtype: object)

#### Now look for differing address again

In [149]:
def lev_list(address_list, address,i):
    temp_list = []
    temp_list_u = []
    for u_address in address_list:
        if Levenshtein.distance(address, u_address.upper()) <= i:
            temp_list.append(address)
            temp_list_u.append(u_address.upper())
    temp_list_series = pd.Series(temp_list)
    temp_list_u_series = pd.Series(temp_list_u)
    return temp_list_u_series

In [150]:
def space_fix_utilities(s_util):
    s_util_to_series = []
    for address in s_util:
        full_address = address.split()
        length = len(full_address)
        word = full_address[0] + "  "
        for i in range(1, length-1):
            word += full_address[i] + " "
        word += " " + full_address[length-1]
        word = word.strip()
        s_util_to_series.append(word)
    return pd.Series(s_util_to_series)

In [151]:
def construct_fixed_addresses(list_series):
    tmp_to_series = []
    for (apt, dup, sno, snm, stp) in zip(list_series[0],list_series[1],list_series[2],
                                         list_series[3],list_series[4]):
        word = ""
        if sno != "NA":
            word += sno + "  "
        if snm != "NA":
            word += snm + "  "
        if stp != "NA":
            word += stp
        tmp_to_series.append(word.strip())
    return pd.Series(tmp_to_series)

In [152]:
s_utilities = series_utilities.unique()

In [153]:
#s_utilities = df_utilities2018['Premise Address']
s_fixed_utilities = space_fix_utilities(s_utilities)
s_fixed_home = construct_fixed_addresses(home)
s_fixed_cdbg = construct_fixed_addresses(cdbg)

In [154]:
lev_list(s_fixed_utilities,"2226  OLD DOMINION  DR", 2);

#### These are addresses that are still not reconciled with utilities

In [155]:
mismatched_home = s_fixed_home[~(s_fixed_home.isin(s_fixed_utilities.str.upper()))]

In [156]:
mismatched_cdbg = s_fixed_cdbg[~(s_fixed_cdbg.isin(s_fixed_utilities.str.upper()))]

In [157]:
mismatched_home

57             2620  S MADISON  ST
84               529  COCHRAN  AVE
86          2226  OLD DOMINION  DR
87             2701  DONCASTER  RD
88            250  BONNY VIEW  AVE
89            249  BONNY VIEW  AVE
90            248  BONNY VIEW  AVE
91            251  BONNY VIEW  AVE
92            253  BONNY VIEW  AVE
93            247  BONNY VIEW  AVE
94            254  BONNY VIEW  AVE
95            243  BONNY VIEW  AVE
96            245  BONNY VIEW  AVE
97            252  BONNY VIEW  AVE
111            2010  LAFAYETTE  CR
116              20  S JACKSON  ST
117               603  N HALEY  ST
118           1112  PEACHTREE  TER
130             2517  CARDINAL  DR
138            2006  LAFAYETTE  CR
145              818  WESTVIEW  DR
148              2501  REDWOOD  CR
167              933  RANDOLPH  DR
173            2201  CHAMPAGNE  DR
188              215  BROADWAY  ST
194              1003  HOLLAND  DR
206            2200  CHAMPAGNE  LN
211           1019  UNIVERSITY  DR
212            509  

In [101]:
mismatched_home.to_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/mismatched_home.csv', index = False, header = False)
mismatched_cdbg.to_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Raw-Data/mismatched_cdbg.csv', index = False)

/anaconda3/envs/cds/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  


In [109]:
m_test.iloc[:,0]

0             2620  S MADISON  ST
1               529  COCHRAN  AVE
2          2226  OLD DOMINION  DR
3             2701  DONCASTER  RD
4            250  BONNY VIEW  AVE
5            249  BONNY VIEW  AVE
6            248  BONNY VIEW  AVE
7            251  BONNY VIEW  AVE
8            253  BONNY VIEW  AVE
9            247  BONNY VIEW  AVE
10           254  BONNY VIEW  AVE
11           243  BONNY VIEW  AVE
12           245  BONNY VIEW  AVE
13           252  BONNY VIEW  AVE
14            2010  LAFAYETTE  CR
15              20  S JACKSON  ST
16               603  N HALEY  ST
17           1112  PEACHTREE  TER
18             2517  CARDINAL  DR
19            2006  LAFAYETTE  CR
20              818  WESTVIEW  DR
21              2501  REDWOOD  CR
22              933  RANDOLPH  DR
23            2201  CHAMPAGNE  DR
24              215  BROADWAY  ST
25              1003  HOLLAND  DR
26            2200  CHAMPAGNE  LN
27           1019  UNIVERSITY  DR
28            509  WESTOVER  BLVD
29            

In [80]:
mismatched_cdbg

41                         UNKNOWN ADDRESS
48                          400  EBONY  LN
50                    638  W HIGHLAND  AVE
51                         UNKNOWN ADDRESS
52                         UNKNOWN ADDRESS
54                         UNKNOWN ADDRESS
93                  230  S JACKSON ST  250
101                        UNKNOWN ADDRESS
131                        UNKNOWN ADDRESS
141                        UNKNOWN ADDRESS
143                       302  S CASON  ST
144                        UNKNOWN ADDRESS
152                     SUPPRESSED ADDRESS
156                     225  S JACKSON  ST
159                   1010  W LINCOLN  AVE
167    1401  E CAMPBELL ST/108 CARROLL  ST
191                     2572  CRESCENT  DR
192                     2572  CRESCENT  DR
208                       4835  IMPALA  LN
210                         507  EMILY  ST
213                    608  RESIDENCE  AVE
214                       614  ROADWAY  ST
216                       706  E TOWNE  RD
231        

In [68]:
df_all_addresses = pd.read_csv('~/Dropbox (Amherst College)/CDS-2019-AlbanyHub/Processed-Data/all_addresses.csv')
df_all_addresses = df_all_addresses.drop('Unnamed: 0', axis = 1)

In [69]:
df_all_addresses.head()
s_all_addresses = df_all_addresses['line1']

In [70]:
lev_list(s_all_addresses,"250 BONNY VIEW AVE", 2)

Series([], dtype: float64)

In [71]:
def gen_list_none(address_list, mismatched, i):
    tmp_list = []
    for address in mismatched:
        if address == "UNKNOWN ADDRESS" or address == "SUPPRESSED ADDRESS":
            continue
        if lev_list(address_list, address, i).shape[0] == 0:
            tmp_list.append(address)
    return pd.Series(tmp_list)

In [72]:
gen_list_none(s_all_addresses,mismatched_home, 3)

0            250  BONNY VIEW  AVE
1            249  BONNY VIEW  AVE
2            248  BONNY VIEW  AVE
3            251  BONNY VIEW  AVE
4            253  BONNY VIEW  AVE
5            247  BONNY VIEW  AVE
6            254  BONNY VIEW  AVE
7            243  BONNY VIEW  AVE
8            245  BONNY VIEW  AVE
9            252  BONNY VIEW  AVE
10            2010  LAFAYETTE  CR
11             2517  CARDINAL  DR
12            2006  LAFAYETTE  CR
13              933  RANDOLPH  DR
14            2201  CHAMPAGNE  DR
15              215  BROADWAY  ST
16            2200  CHAMPAGNE  LN
17           1019  UNIVERSITY  DR
18           2710  W OAKRDIGE  DR
19    1404  WHISPERING PINES  AVE
dtype: object

In [73]:
gen_list_none(s_all_addresses,mismatched_cdbg, 3)

0                  230  S JACKSON ST  250
1                        302  S CASON  ST
2     1401  E CAMPBELL ST/108 CARROLL  ST
3                          507  EMILY  ST
4                        614  ROADWAY  ST
5                        706  E TOWNE  RD
6                         743  E TOWN  RD
7                   215  BROADWAY CT  #13
8                   215  BROADWAY CT  #10
9                 255  E OGLETHORPE  BLVD
10                259  E OGLETHORPE  BLVD
11               1817  W OGLETHORPE  BLVD
12                      1819  EDGERLY  DR
13                      1921  EDGERLY  DR
14                     214  N BROADAY  ST
15                       2301  GEORGE  ST
16                 1934  S RIVERVIEW  CIR
17                  2028  N RIVERVIEW  DR
18              9001300  BLOCK MOBILE  RD
19              700  BLOCK NETHERLAND  LN
20               405  JOHNNY WILLIAMS  RD
dtype: object

In [449]:
lev_list(s_fixed_utilities,"2620  S MADISON  ST", 1)

0      2624  S MADISON  ST
1      2624  S MADISON  ST
2      2622  S MADISON  ST
3      2622  S MADISON  ST
4      2622  S MADISON  ST
5      2220  S MADISON  ST
6      2220  S MADISON  ST
7      2220  S MADISON  ST
8      2220  S MADISON  ST
9      2220  S MADISON  ST
10     2220  S MADISON  ST
11     2220  S MADISON  ST
12     2220  S MADISON  ST
13     2220  S MADISON  ST
14     2220  S MADISON  ST
15     2220  S MADISON  ST
16     2120  S MADISON  ST
17     2120  S MADISON  ST
18     2120  S MADISON  ST
19     2120  S MADISON  ST
20     2120  S MADISON  ST
21     2020  S MADISON  ST
22     2020  S MADISON  ST
23     2020  S MADISON  ST
24     2020  S MADISON  ST
25     2020  S MADISON  ST
26     2020  S MADISON  ST
27     2020  S MADISON  ST
28     2020  S MADISON  ST
29     2020  S MADISON  ST
30     2020  S MADISON  ST
31     2020  S MADISON  ST
32     2020  S MADISON  ST
33     2624  S MADISON  ST
34     2624  S MADISON  ST
35     2624  S MADISON  ST
36     2624  S MADISON  ST
3

In [67]:
mismatched_home.shape[0] + mismatched_cdbg.shape[0]

99

In [462]:
s_u_unique = s_fixed_utilities.unique()

In [452]:
df_all_addresses.shape

(37940, 14)

In [453]:
29398 / 37940

0.7748550342646283

In [454]:
850 * .77

654.5

In [467]:
gen_list_none(s_u_unique, mismatched_home, 2)

0              818  WESTVIEW  DR
1            1005  E FOURTH  AVE
2       421  N WESTOVER BLVD  18
3    1404  WHISPERING PINES  AVE
dtype: object

In [468]:
gen_list_none(s_u_unique, mismatched_cdbg, 2)

0                  230  S JACKSON ST  250
1     1401  E CAMPBELL ST/108 CARROLL  ST
2                        4835  IMPALA  LN
3                        706  E TOWNE  RD
4                         743  E TOWN  RD
5                   215  BROADWAY CT  #13
6                   215  BROADWAY CT  #10
7                       1819  EDGERLY  DR
8                       1921  EDGERLY  DR
9                        2301  GEORGE  ST
10              9001300  BLOCK MOBILE  RD
11              700  BLOCK NETHERLAND  LN
12               405  JOHNNY WILLIAMS  RD
dtype: object

In [480]:
lev_list(s_u_unique,"2226  OLD DOMINION  DR", 3)

0    2225  OLD DOMINION  RD
1    2221  OLD DOMINION  RD
2    2223  OLD DOMINION  RD
3    2216  OLD DOMINION  RD
4    2222  OLD DOMINION  RD
5    2220  OLD DOMINION  RD
6    2224  OLD DOMINION  RD
7    2226  OLD DOMINION  RD
8    2228  OLD DOMINION  RD
9    2206  OLD DOMINION  RD
dtype: object

In [476]:
mismatched_home

57             2620  S MADISON  ST
74                 1605  SUNNY  LN
84               529  COCHRAN  AVE
86          2226  OLD DOMINION  DR
87             2701  DONCASTER  RD
88            250  BONNY VIEW  AVE
89            249  BONNY VIEW  AVE
90            248  BONNY VIEW  AVE
91            251  BONNY VIEW  AVE
92            253  BONNY VIEW  AVE
93            247  BONNY VIEW  AVE
94            254  BONNY VIEW  AVE
95            243  BONNY VIEW  AVE
96            245  BONNY VIEW  AVE
97            252  BONNY VIEW  AVE
111            2010  LAFAYETTE  CR
116              20  S JACKSON  ST
117               603  N HALEY  ST
118           1112  PEACHTREE  TER
130             2517  CARDINAL  DR
138            2006  LAFAYETTE  CR
145              818  WESTVIEW  DR
148              2501  REDWOOD  CR
167              933  RANDOLPH  DR
173            2201  CHAMPAGNE  DR
188              215  BROADWAY  ST
190            1005  E FOURTH  AVE
194              1003  HOLLAND  DR
206            2200 

In [477]:
df1_fixed.iloc[mismatched_home.index]

,Parcel ID,Address,City,State,XY Coordinates,Plan Year,IDIS Project,Project,IDIS Activity ID,Activity Status,Program,Funded Amount,Draw Amount,Balance
57,00012/00004/011,2620 S MADISON ST,Albany,GA,"31.5319347, -84.1596916",2010,22,Acquisition/Rehabilitation,928,Completed,HOME,53553.46,53553.46,0.00
74,000HH/00061/011,1605 SUNNY LN,Albany,GA,"31.558721, -84.1635551",2011,21,New Construction,996,Completed,HOME,56803.00,56803.00,0.00
84,000BB/00035/015,529 COCHRAN AVE,Albany,GA,"31.546907, -84.159568",2012,10,Tenant Based Rental Assistance,1083,Completed,HOME,12681.00,12681.00,0.00
86,044-2/00004/034,2226 B OLD DOMINION DR,Albany,GA,"31.59829, -84.2457838",2012,10,Tenant Based Rental Assistance,1099,Completed,HOME,2208.53,2208.53,0.00
87,0000S/00010/006,2701 B DONCASTER RD,Albany,GA,"31.6143574, -84.1982744",2012,10,Tenant Based Rental Assistance,1084,Completed,HOME,4916.00,4916.00,0.00
88,0000C/00005/026,250 BONNY VIEW AVE,Albany,GA,"31.577898, -84.1210591",2012,20,New Construction,1116,Completed,HOME,158828.16,158828.16,0.00
89,0000C/00005/029,249 BONNY VIEW AVE,Albany,GA,"31.577898, -84.1210591",2012,20,New Construction,1117,Completed,HOME,158828.16,158828.16,0.00
90,0000C/00005/026,248 BONNY VIEW AVE,Albany,GA,"31.577898, -84.1210591",2012,20,New Construction,1118,Completed,HOME,158828.16,158828.16,0.00
91,0000C/00005/030,251 BONNY VIEW AVE,Albany,GA,"31.577898, -84.1210591",2012,20,New Construction,1119,Completed,HOME,158828.16,158828.16,0.00
92,0000C/00005/030,253 BONNY VIEW AVE,Albany,GA,"31.577898, -84.1210591",2012,20,New Construction,1120,Completed,HOME,158828.16,158828.16,0.00


#### Thoughts thus far

It looks like there is some (tiny) amounts of hand cleaning left to do. <br>
More importantly, there seem to be 121 addresses that have not yet been reconciled with the utilities data. Of those 121, there are (approximately) 40 addresses of which there seem to be nothing even close in the full address set. Of those same 121, there are (approximately) 10 address of which there seem to be nothing even close in the utilities data. This is a bit of a strange contradiction, considering the full address set is several thousand records more than the utilities data set. There is a large possibility that the utilities data does not necessarily cover all the addresses in Albany. <br>
What does work flow look like moving on? <br>
Goal: have 0 or very small amounts of mismatched.
Go through each 121 addresses and look at if there are any addresses that are relatively close, that could have been typos. Or check to see if they were demolitions. <br>
Go through the remaining addresses and ask shuronda and dianne about whether they are typos? <br>